In [1]:
pip install spade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 387 kB 7.5 MB/s 
     |████████████████████████████████| 131 kB 41.1 MB/s 
     |████████████████████████████████| 309 kB 11.5 MB/s 
     |████████████████████████████████| 1.3 MB 47.6 MB/s 
     |████████████████████████████████| 133 kB 55.9 MB/s 
     |████████████████████████████████| 76 kB 6.3 MB/s 
     |████████████████████████████████| 271 kB 54.8 MB/s 
     |████████████████████████████████| 64 kB 3.3 MB/s 
     |████████████████████████████████| 269 kB 51.1 MB/s 
     |████████████████████████████████| 55 kB 4.8 MB/s 
     |████████████████████████████████| 4.0 MB 42.1 MB/s 
  Created wheel for aioxmpp: filename=aioxmpp-0.13.2-py3-none-any.whl size=471415 sha256=b15a24f42930a6b00f24cad76ce81c66a3575a9b0f89e1706e879b05c3f21bc1
  Stored in directory: /root/.cache/pip/wheels/81/0c/86/a64dbae6e3f03010fa95e255b50c69c0f059857941438fa174
  Created

In [4]:
# Для очистки консоли
from IPython.display import clear_output

In [5]:
import datetime
import getpass
import time

from spade import quit_spade
from spade.agent import Agent
from spade.behaviour import CyclicBehaviour, PeriodicBehaviour
from spade.message import Message
from spade.behaviour import OneShotBehaviour

In [6]:
agents_list = dict()
agents_is_losed = dict()

In [7]:
class CoordinatorAgent(Agent):
    
    class ListenBehav(PeriodicBehaviour):
        """
        Информування про роботу агенту координатору
        """
        def clock(self):
            # print(f"Агент {self.agent.jid} работает")
            clock_counter = self.get("clock_counter")
            if clock_counter is None:
                clock_counter = 0
            else:
                clock_counter %= 10
                if clock_counter == 0:
                    print(f"{datetime.datetime.now()}: 'I'm OK'")
                clock_counter+=1
            self.set("clock_counter", clock_counter)
        async def run(self):
            # print(f"Агент {self.agent.jid} работает")
            self.clock()
            msg = await self.receive(timeout=1)

            if msg is not None:
                sender = f"{msg.sender.localpart}@{msg.sender.domain}"
                print(f"Агент {self.agent.jid} услышал {msg.body} from {sender}")
                if msg.metadata['msg_type'] is not None:
                    # print("is not None yeah")
                    msg_type = msg.metadata['msg_type']
                    # print(f"is {msg_type}")
                    if msg_type == "connect":
                        answer = self.addAgent(sender, msg.body)
                        # if answer:
                        await self.sendTable(sender)
                            # delll
                            # time.sleep(5)
                    elif (msg_type == "Update"):
                        if sender not in list(agents_list.keys()):
                            self.addAgent(sender, msg.body)
                        await self.sendTable(sender)
                    # if msg_type == "disconnect":
                    #     self.dellAgent(sender)
                    #     print("dc yeah")
                    

        async def sendTable(self, sender):
            msg = Message(to=sender)
            msg.set_metadata("performative", "inform")  # Set the "inform" FIPA performative
            msg.set_metadata("ontology", "myOntology")  # Set the ontology of the message content
            msg.set_metadata("language", "OWL-S")       # Set the language of the message content
            msg.set_metadata("msg_type", "connectTable")
            msg.body = str(agents_list)
            print(msg.body)
            await self.send(msg)

        def dellAgent(self, sender):
            if len(agents_list) != 0:
                if (sender in agents_list):
                    agents_list.pop(sender)
                    return True
                return False
            else:
                return False

        # Add typeAgent
        def addAgent(self, sender, agnt_type):
            # if len(agents_list) == 0:
                agents_list.update({sender: agnt_type})
                agents_is_losed.update({sender: 0})


    class MonitorActiveBeh(PeriodicBehaviour):
        async def run(self):
            cnt = 0
           
            while cnt <= 7:
                msg = await self.receive(timeout=0.5)

                if msg is not None:
                    sender = f"{msg.sender.localpart}@{msg.sender.domain}"
                    print(f"Агент {self.agent.jid} услышал {msg.body} from {sender}")
                    if msg.metadata['msg_type'] is not None:
                        msg_type = msg.metadata['msg_type']
                        if (msg_type == "pong"):
                            agents_is_losed[sender] = 0
                            print("Счётчик простоя обнулился")
                            cnt = 0
                        # else: cnt =+ 1
                cnt =+ 1

    class PingBeh(PeriodicBehaviour):
        async def run(self):
            removed_list = list()
            for i in agents_is_losed:
                agents_is_losed[i] += 1
                await self.ping(i)
                if agents_is_losed[i] >= 7:
                    # print(agents_is_losed)
                    # print(agents_list)
                    removed_list.append(i)
                    # time.sleep(5.5)
                    # agents_is_losed.pop(i)
                    # agents_list.pop(i)
                    # pass
            for i in removed_list:
                print(agents_is_losed)
                print(agents_list)
                time.sleep(3)
                agents_is_losed.pop(i)
                agents_list.pop(i)
                print(f"Agent {i} remowed from list")

        async def ping(self, _to):
            print(f"ping {_to}")
            msg = Message(to=_to)     # Instantiate the message
            msg.set_metadata("performative", "inform")  # Set the "inform" FIPA performative
            msg.set_metadata("ontology", "myOntology")  # Set the ontology of the message content
            msg.set_metadata("language", "OWL-S")       # Set the language of the message content
            msg.body = "PING"                    # Set the message content
            msg.set_metadata("msg_type", "ping")
            await self.send(msg)

    async def setup(self):
        print("CoordinatorAgent started")
        start_at = datetime.datetime.now() + datetime.timedelta(seconds=1)
        listenBeh = self.ListenBehav(period=1.3,start_at = start_at)
        monitorBeh = self.MonitorActiveBeh(period=5,start_at = start_at)
        pingBeh = self.PingBeh(period=5,start_at = start_at)
        self.add_behaviour(listenBeh)
        self.add_behaviour(monitorBeh)
        self.add_behaviour(pingBeh)


In [10]:
coord = CoordinatorAgent("your_jid@your_xmpp_server", "your_password")

future = coord.start(auto_register=True)
future.result()

ans = ""
while (ans != "ex"):
    ans = input()
    coord.stop()

print("программа завершила работу")

ex
программа завершила работу


'Factory'

In [ ]:
agents_list

{}